In [54]:
from google.colab import files
uploaded = files.upload()

Saving ArticlesApril2017.csv to ArticlesApril2017.csv


In [55]:
import pandas as pd
import tensorflow as tf

# Path ke file langsung
file_path = '/content/ArticlesApril2017.csv'

# Baca CSV dan ambil kolom headline
headlines_df = pd.read_csv(file_path)
all_headlines = list(headlines_df.headline.values)

print(f"Jumlah headline: {len(all_headlines)}")
print(all_headlines[:5])

Jumlah headline: 886
['Finding an Expansive View  of a Forgotten People in Niger', 'And Now,  the Dreaded Trump Curse', 'Venezuela’s Descent Into Dictatorship', 'Stain Permeates Basketball Blue Blood', 'Taking Things for Granted']


In [56]:
all_headlines[:20]

['Finding an Expansive View  of a Forgotten People in Niger',
 'And Now,  the Dreaded Trump Curse',
 'Venezuela’s Descent Into Dictatorship',
 'Stain Permeates Basketball Blue Blood',
 'Taking Things for Granted',
 'The Caged Beast Awakens',
 'An Ever-Unfolding Story',
 'O’Reilly Thrives as Settlements Add Up',
 'Mouse Infestation',
 'Divide in G.O.P. Now Threatens Trump Tax Plan',
 'Variety Puzzle: Acrostic',
 'They Can Hit a Ball 400 Feet. But Play Catch? That’s Tricky.',
 'In Trump Country, Shock at Trump Budget Cuts',
 'Why Is This Hate Different From All Other Hate?',
 'Pick Your Favorite Ethical Offender',
 'My Son’s Growing Black Pride',
 'Jerks and the Start-Ups They Ruin',
 'Trump  Needs  a Brain',
 'Manhood in the Age of Trump',
 'The Value of a Black College']

In [57]:
# Remove all headlines with the value of "Unknown"
all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

831

In [58]:
all_headlines[:20]

['Finding an Expansive View  of a Forgotten People in Niger',
 'And Now,  the Dreaded Trump Curse',
 'Venezuela’s Descent Into Dictatorship',
 'Stain Permeates Basketball Blue Blood',
 'Taking Things for Granted',
 'The Caged Beast Awakens',
 'An Ever-Unfolding Story',
 'O’Reilly Thrives as Settlements Add Up',
 'Mouse Infestation',
 'Divide in G.O.P. Now Threatens Trump Tax Plan',
 'Variety Puzzle: Acrostic',
 'They Can Hit a Ball 400 Feet. But Play Catch? That’s Tricky.',
 'In Trump Country, Shock at Trump Budget Cuts',
 'Why Is This Hate Different From All Other Hate?',
 'Pick Your Favorite Ethical Offender',
 'My Son’s Growing Black Pride',
 'Jerks and the Start-Ups They Ruin',
 'Trump  Needs  a Brain',
 'Manhood in the Age of Trump',
 'The Value of a Black College']

In [59]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=' ',
    char_level=False,
    oov_token=None
)

In [60]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenize the words in our headlines
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_headlines)
total_words = len(tokenizer.word_index) + 1
print('Total words: ', total_words)

Total words:  2481


In [61]:
# Print a subset of the word_index dictionary created by Tokenizer
subset_dict = {key: value for key, value in tokenizer.word_index.items() \
               if key in ['a','man','a','plan','a','canal','panama']}
print(subset_dict)

{'a': 2, 'plan': 52, 'man': 120}


In [62]:
tokenizer.texts_to_sequences(['a','man','a','plan','a','canal','panama'])

[[2], [120], [2], [52], [2], [], []]

In [63]:
# Convert data to sequence of tokens
input_sequences = []
for line in all_headlines:
    # Convert our headline into a sequence of tokens
    token_list = tokenizer.texts_to_sequences([line])[0]

    # Create a series of sequences for each headline
    for i in range(1, len(token_list)):
        partial_sequence = token_list[:i+1]
        input_sequences.append(partial_sequence)

print(tokenizer.sequences_to_texts(input_sequences[:5]))
input_sequences[:5]

['finding an', 'finding an expansive', 'finding an expansive view', 'finding an expansive view of', 'finding an expansive view of a']


[[179, 20],
 [179, 20, 681],
 [179, 20, 681, 379],
 [179, 20, 681, 379, 4],
 [179, 20, 681, 379, 4, 2]]

In [64]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Determine max sequence length
max_sequence_len = max([len(x) for x in input_sequences])

# Pad all sequences with zeros at the beginning to make them all max length
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0, 179,  20], dtype=int32)

In [65]:
# Predictors are every word except the last
predictors = input_sequences[:,:-1]
# Labels are the last word
labels = input_sequences[:,-1]
labels[:5]

array([ 20, 681, 379,   4,   2], dtype=int32)

In [66]:
from tensorflow.keras import utils

labels = utils.to_categorical(labels, num_classes=total_words)

In [67]:
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=100))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [68]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense

model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=100))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [69]:
model.build(input_shape=(None, 100))
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,329,473 (5.07 MB)

 Trainable params: 1,329,473 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [70]:
model.add(Dense(2481, activation='softmax'))

In [71]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [72]:
model.fit(predictors, labels, epochs=30, verbose=1)

Epoch 1/30
155/155 ━━━━━━━━━━━━━━━━━━━━ 9s 31ms/step - accuracy: 0.0256 - loss: 7.7898
Epoch 2/30
155/155 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - accuracy: 0.0312 - loss: 7.6744
Epoch 3/30
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.0302 - loss: 7.5970
Epoch 4/30
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.0368 - loss: 7.5195
Epoch 5/30
155/155 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - accuracy: 0.0314 - loss: 7.4583
Epoch 6/30
155/155 ━━━━━━━━━━━━━━━━━━━━ 9s 33ms/step - accuracy: 0.0328 - loss: 7.3980
Epoch 7/30
155/155 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - accuracy: 0.0342 - loss: 7.3197
Epoch 8/30
155/155 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - accuracy: 0.0306 - loss: 7.3000
Epoch 9/30
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.0322 - loss: 7.2262
Epoch 10/30
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.0367 - loss: 7.1825
Epoch 11/30
155/155 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - accuracy: 0.0315 - loss: 7.1534
Epoch 12/30
155/155 ━━━━━━━━━━━━━━━━━━━━

In [73]:
import numpy as np

def predict_next_token(seed_text):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list, verbose=0)
    predicted_class = np.argmax(predicted_probs, axis=-1)
    return predicted_class[0]  # hasil berupa 1 angka token ID

In [74]:
prediction = predict_next_token("today in new york")

In [75]:
predicted_word = tokenizer.index_word[prediction]
print(predicted_word)

the


In [76]:
seed_text = "today in new york"
next_word = predict_next_token(seed_text)
full_sentence = seed_text + " " + tokenizer.index_word[next_word]
print(full_sentence)

today in new york the


In [77]:
def generate_headline(seed_text, next_words=1):
    for _ in range(next_words):
        # Predict next token
        prediction = predict_next_token(seed_text)
        # Convert token to word
        next_word = tokenizer.sequences_to_texts([[prediction]])[0]

        # Add next word to the headline. This headline will be used in the next pass of the loop.
        seed_text += " " + next_word
    # Return headline as title-case
    return seed_text.title()

In [78]:
predicted_word = tokenizer.index_word.get(next_word, '')

In [79]:
seed_texts = [
    'washington dc is',
    'today in new york',
    'the school district has',
    'crime has become']
for seed in seed_texts:
    print(generate_headline(seed, next_words=5))

Washington Dc Is The The The The The
Today In New York The The The The The
The School District Has The The The The The
Crime Has Become The The The The The


In [80]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}